# Imports

In [18]:
import duckdb
import h3
import pandas as pd
from keplergl import KeplerGl

In [19]:
with open('secret_access_key.txt', 'r') as file:
    secret_access_key = file.read().strip()

with open('access_key.txt', 'r') as file:
    access_key = file.read().strip()

In [23]:
%%time
# Connect to DuckDB
db = duckdb.connect()

# Enable S3 access
db.execute("INSTALL httpfs;")
db.execute("LOAD httpfs;")

CPU times: user 150 ms, sys: 16.2 ms, total: 167 ms
Wall time: 47.6 ms


In [25]:
%%time
db.execute("SET s3_region='us-east-1';")  # Adjust if needed
db.execute("SET s3_endpoint='https://data.source.coop';")  # Custom endpoint
db.execute(f"SET s3_access_key_id={access_key};")
db.execute(f"SET s3_secret_access_key={secret_access_key};")

# Corrected S3 path
s3_path = "s3://wherobots/usa-structures/buildings.parquet"

CPU times: user 751 μs, sys: 604 μs, total: 1.36 ms
Wall time: 706 μs


In [26]:
%%time
# Define the correct S3 path
s3_path = "s3://wherobots/usa-structures/buildings.parquet"

# Query a subset of the data
query = f"""
    SELECT latitude, longitude, building_id
    FROM read_parquet('{s3_path}')
    WHERE latitude IS NOT NULL AND longitude IS NOT NULL
    LIMIT 10000
"""

# Execute query
buildings_df = db.query(query).to_df()
print(buildings_df.head())  # Preview results

IOException: IO Error: Could not establish connection error for HTTP HEAD to 'https://wherobots.https://data.source.coop/usa-structures/buildings.parquet'

In [ ]:
# Convert lat/lon to H3 indices at resolution 9
buildings_df['h3_index'] = buildings_df.apply(
    lambda row: h3.geo_to_h3(row['latitude'], row['longitude'], 9), axis=1
)

# Save as GeoJSON-compatible DataFrame
kepler_map = KeplerGl()
kepler_map.add_data(data=buildings_df, name="Buildings")
kepler_map

In [9]:
%%time
# Query a subset of the data (modify as needed)
query = f"""
    SELECT latitude, longitude, building_id
    FROM read_parquet('{s3_path}')
    WHERE latitude IS NOT NULL AND longitude IS NOT NULL
    LIMIT 10000
"""

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 3.81 μs


In [10]:
%%time
# Execute query
buildings_df = db.query(query).to_df()

HTTPException: HTTP Error: HTTP GET error on 'https://wherobots.s3.amazonaws.com/usa-structures/' (HTTP 403)

In [ ]:
%%time
# Convert lat/lon to H3 indices at resolution 9
buildings_df['h3_index'] = buildings_df.apply(
    lambda row: h3.geo_to_h3(row['latitude'], row['longitude'], 9), axis=1
)

In [ ]:
%%time
# Save as GeoJSON-compatible DataFrame
kepler_map = KeplerGl()
kepler_map.add_data(data=buildings_df, name="Buildings")
kepler_map